In [0]:
# ============================================
# ENHANCED INSTACART ASSOCIATION ANALYSIS
# Pure SQL Implementation (No ML Library - Spark Connect Compatible)
# ============================================

from pyspark.sql.functions import col, count, collect_list, collect_set, round as spark_round
import pandas as pd

# NOTE: This notebook uses PURE SQL queries - no Spark ML libraries
# Compatible with Databricks Spark Connect

print("Starting Market Basket Analysis - Pure SQL Implementation")
print("="*70)

# ============================================
# DATA QUALITY CHECK
# ============================================

def check_data_quality():
    """
    Check for data quality issues in the products table
    """
    print("\n→ Checking data quality...")
    
    quality_check = spark.sql("""
        SELECT 
            COUNT(*) as total_products,
            COUNT(CASE WHEN TRY_CAST(department_id AS BIGINT) IS NULL THEN 1 END) as invalid_dept_ids,
            COUNT(CASE WHEN TRY_CAST(aisle_id AS BIGINT) IS NULL THEN 1 END) as invalid_aisle_ids,
            COUNT(CASE WHEN TRY_CAST(department_id AS BIGINT) IS NOT NULL 
                       AND TRY_CAST(aisle_id AS BIGINT) IS NOT NULL THEN 1 END) as valid_products
        FROM workspace.instacart.products
    """)
    
    result = quality_check.collect()[0]
    print(f"   Total Products: {result['total_products']}")
    print(f"   Invalid Department IDs: {result['invalid_dept_ids']}")
    print(f"   Invalid Aisle IDs: {result['invalid_aisle_ids']}")
    print(f"   Valid Products: {result['valid_products']}")
    
    if result['invalid_dept_ids'] > 0 or result['invalid_aisle_ids'] > 0:
        print("   ⚠ Some products have invalid IDs - they will be excluded from department/aisle analysis")
    
    return result

# Run data quality check
check_data_quality()

print("="*70)

# ============================================
# PART 1: HIGH-IMPACT PRODUCT PAIRS
# ============================================

def analyze_product_pairs(min_support_count=100):
    """
    Find product pairs with support, confidence, and lift metrics
    """
    print("\n→ Analyzing product pairs...")
    
    query = f"""
    WITH product_pairs AS (
        SELECT 
            op1.product_id as prod1,
            op2.product_id as prod2,
            COUNT(DISTINCT op1.order_id) as co_occurrence
        FROM workspace.instacart.order_products_prior op1
        JOIN workspace.instacart.order_products_prior op2 
            ON op1.order_id = op2.order_id AND op1.product_id < op2.product_id
        GROUP BY op1.product_id, op2.product_id
        HAVING COUNT(DISTINCT op1.order_id) >= {min_support_count}
    ),
    product_support AS (
        SELECT 
            product_id,
            COUNT(DISTINCT order_id) as support_count
        FROM workspace.instacart.order_products_prior
        GROUP BY product_id
    ),
    total_orders AS (
        SELECT COUNT(DISTINCT order_id) as total 
        FROM workspace.instacart.order_products_prior
    )
    SELECT 
        p1.product_name as product_1,
        p2.product_name as product_2,
        pp.co_occurrence,
        ROUND(pp.co_occurrence * 100.0 / to.total, 4) as support_pct,
        ROUND(pp.co_occurrence * 1.0 / ps1.support_count, 4) as confidence_1_to_2,
        ROUND(pp.co_occurrence * 1.0 / ps2.support_count, 4) as confidence_2_to_1,
        ROUND(pp.co_occurrence * 1.0 * to.total / (ps1.support_count * ps2.support_count), 2) as lift,
        d1.department as dept_1,
        d2.department as dept_2,
        a1.aisle as aisle_1,
        a2.aisle as aisle_2
    FROM product_pairs pp
    CROSS JOIN total_orders to
    JOIN workspace.instacart.products p1 ON pp.prod1 = p1.product_id
    JOIN workspace.instacart.products p2 ON pp.prod2 = p2.product_id
    JOIN product_support ps1 ON pp.prod1 = ps1.product_id
    JOIN product_support ps2 ON pp.prod2 = ps2.product_id
    LEFT JOIN workspace.instacart.departments d1 ON TRY_CAST(p1.department_id AS BIGINT) = d1.department_id
    LEFT JOIN workspace.instacart.departments d2 ON TRY_CAST(p2.department_id AS BIGINT) = d2.department_id
    LEFT JOIN workspace.instacart.aisles a1 ON TRY_CAST(p1.aisle_id AS BIGINT) = a1.aisle_id
    LEFT JOIN workspace.instacart.aisles a2 ON TRY_CAST(p2.aisle_id AS BIGINT) = a2.aisle_id
    ORDER BY lift DESC
    LIMIT 200
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("product_pairs_analysis")
    return df


# ============================================
# PART 2: TRIPLET ANALYSIS
# ============================================

def analyze_triplets(min_support_count=50):
    """
    Find 3-item combinations that frequently appear together
    """
    print("\n→ Analyzing triplets (3-item sets)...")
    
    query = f"""
    WITH triplet_combos AS (
        SELECT 
            op1.product_id as prod1,
            op2.product_id as prod2,
            op3.product_id as prod3,
            COUNT(DISTINCT op1.order_id) as triplet_count
        FROM workspace.instacart.order_products_prior op1
        JOIN workspace.instacart.order_products_prior op2 
            ON op1.order_id = op2.order_id AND op1.product_id < op2.product_id
        JOIN workspace.instacart.order_products_prior op3
            ON op1.order_id = op3.order_id AND op2.product_id < op3.product_id
        GROUP BY op1.product_id, op2.product_id, op3.product_id
        HAVING COUNT(DISTINCT op1.order_id) >= {min_support_count}
    ),
    total_orders AS (
        SELECT COUNT(DISTINCT order_id) as total 
        FROM workspace.instacart.order_products_prior
    )
    SELECT 
        p1.product_name as product_1,
        p2.product_name as product_2,
        p3.product_name as product_3,
        tc.triplet_count,
        ROUND(tc.triplet_count * 100.0 / to.total, 4) as support_pct,
        d1.department as dept_1,
        d2.department as dept_2,
        d3.department as dept_3,
        CASE 
            WHEN d1.department = d2.department AND d2.department = d3.department 
            THEN 'Same Department'
            WHEN d1.department != d2.department AND d2.department != d3.department AND d1.department != d3.department
            THEN 'All Different Departments'
            ELSE 'Mixed Departments'
        END as department_diversity
    FROM triplet_combos tc
    CROSS JOIN total_orders to
    JOIN workspace.instacart.products p1 ON tc.prod1 = p1.product_id
    JOIN workspace.instacart.products p2 ON tc.prod2 = p2.product_id
    JOIN workspace.instacart.products p3 ON tc.prod3 = p3.product_id
    LEFT JOIN workspace.instacart.departments d1 ON TRY_CAST(p1.department_id AS BIGINT) = d1.department_id
    LEFT JOIN workspace.instacart.departments d2 ON TRY_CAST(p2.department_id AS BIGINT) = d2.department_id
    LEFT JOIN workspace.instacart.departments d3 ON TRY_CAST(p3.department_id AS BIGINT) = d3.department_id
    ORDER BY triplet_count DESC
    LIMIT 100
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("triplet_patterns")
    return df


# ============================================
# PART 3: QUADRUPLET ANALYSIS
# ============================================

def analyze_quadruplets(min_support_count=30):
    """
    Find 4-item combinations - the most complete purchase patterns
    """
    print("\n→ Analyzing quadruplets (4-item sets)...")
    
    query = f"""
    WITH quad_combos AS (
        SELECT 
            op1.product_id as prod1,
            op2.product_id as prod2,
            op3.product_id as prod3,
            op4.product_id as prod4,
            COUNT(DISTINCT op1.order_id) as quad_count
        FROM workspace.instacart.order_products_prior op1
        JOIN workspace.instacart.order_products_prior op2 
            ON op1.order_id = op2.order_id AND op1.product_id < op2.product_id
        JOIN workspace.instacart.order_products_prior op3
            ON op1.order_id = op3.order_id AND op2.product_id < op3.product_id
        JOIN workspace.instacart.order_products_prior op4
            ON op1.order_id = op4.order_id AND op3.product_id < op4.product_id
        GROUP BY op1.product_id, op2.product_id, op3.product_id, op4.product_id
        HAVING COUNT(DISTINCT op1.order_id) >= {min_support_count}
    ),
    total_orders AS (
        SELECT COUNT(DISTINCT order_id) as total 
        FROM workspace.instacart.order_products_prior
    )
    SELECT 
        p1.product_name as product_1,
        p2.product_name as product_2,
        p3.product_name as product_3,
        p4.product_name as product_4,
        qc.quad_count,
        ROUND(qc.quad_count * 100.0 / to.total, 4) as support_pct,
        d1.department as dept_1,
        d2.department as dept_2,
        d3.department as dept_3,
        d4.department as dept_4
    FROM quad_combos qc
    CROSS JOIN total_orders to
    JOIN workspace.instacart.products p1 ON qc.prod1 = p1.product_id
    JOIN workspace.instacart.products p2 ON qc.prod2 = p2.product_id
    JOIN workspace.instacart.products p3 ON qc.prod3 = p3.product_id
    JOIN workspace.instacart.products p4 ON qc.prod4 = p4.product_id
    LEFT JOIN workspace.instacart.departments d1 ON TRY_CAST(p1.department_id AS BIGINT) = d1.department_id
    LEFT JOIN workspace.instacart.departments d2 ON TRY_CAST(p2.department_id AS BIGINT) = d2.department_id
    LEFT JOIN workspace.instacart.departments d3 ON TRY_CAST(p3.department_id AS BIGINT) = d3.department_id
    LEFT JOIN workspace.instacart.departments d4 ON TRY_CAST(p4.department_id AS BIGINT) = d4.department_id
    ORDER BY quad_count DESC
    LIMIT 50
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("quadruplet_patterns")
    return df


# ============================================
# PART 4: TEMPORAL PATTERNS
# ============================================

def analyze_temporal_patterns(min_support=50):
    """
    Shopping patterns by time of day and day of week
    """
    print("\n→ Analyzing temporal patterns...")
    
    query = f"""
    WITH time_patterns AS (
        SELECT 
            CASE 
                WHEN o.order_hour_of_day BETWEEN 6 AND 11 THEN 'Morning (6-11am)'
                WHEN o.order_hour_of_day BETWEEN 12 AND 17 THEN 'Afternoon (12-5pm)'
                WHEN o.order_hour_of_day BETWEEN 18 AND 21 THEN 'Evening (6-9pm)'
                ELSE 'Night (10pm-5am)'
            END as time_period,
            CASE 
                WHEN o.order_dow IN (0, 6) THEN 'Weekend'
                ELSE 'Weekday'
            END as day_type,
            op1.product_id as prod1,
            op2.product_id as prod2,
            COUNT(DISTINCT o.order_id) as pair_count
        FROM workspace.instacart.orders o
        JOIN workspace.instacart.order_products_prior op1 ON o.order_id = op1.order_id
        JOIN workspace.instacart.order_products_prior op2 
            ON o.order_id = op2.order_id AND op1.product_id < op2.product_id
        WHERE o.eval_set = 'prior'
        GROUP BY time_period, day_type, op1.product_id, op2.product_id
        HAVING COUNT(DISTINCT o.order_id) >= {min_support}
    )
    SELECT 
        tp.time_period,
        tp.day_type,
        p1.product_name as product_1,
        p2.product_name as product_2,
        tp.pair_count,
        d1.department as dept_1,
        d2.department as dept_2,
        CONCAT(tp.day_type, ' - ', tp.time_period) as shopping_context
    FROM time_patterns tp
    JOIN workspace.instacart.products p1 ON tp.prod1 = p1.product_id
    JOIN workspace.instacart.products p2 ON tp.prod2 = p2.product_id
    LEFT JOIN workspace.instacart.departments d1 ON TRY_CAST(p1.department_id AS BIGINT) = d1.department_id
    LEFT JOIN workspace.instacart.departments d2 ON TRY_CAST(p2.department_id AS BIGINT) = d2.department_id
    ORDER BY tp.pair_count DESC
    LIMIT 200
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("temporal_patterns")
    return df


# ============================================
# PART 5: CROSS-DEPARTMENT DISCOVERIES
# ============================================

def analyze_cross_department_synergies(min_lift=2.5, min_support=30):
    """
    Unexpected cross-department product combinations
    """
    print("\n→ Finding cross-department synergies...")
    
    query = f"""
    WITH dept_pairs AS (
        SELECT 
            TRY_CAST(p1.department_id AS BIGINT) as dept1_id,
            TRY_CAST(p2.department_id AS BIGINT) as dept2_id,
            op1.product_id as prod1,
            op2.product_id as prod2,
            COUNT(DISTINCT op1.order_id) as pair_count
        FROM workspace.instacart.order_products_prior op1
        JOIN workspace.instacart.order_products_prior op2 
            ON op1.order_id = op2.order_id AND op1.product_id < op2.product_id
        JOIN workspace.instacart.products p1 ON op1.product_id = p1.product_id
        JOIN workspace.instacart.products p2 ON op2.product_id = p2.product_id
        WHERE TRY_CAST(p1.department_id AS BIGINT) IS NOT NULL
          AND TRY_CAST(p2.department_id AS BIGINT) IS NOT NULL
          AND TRY_CAST(p1.department_id AS BIGINT) != TRY_CAST(p2.department_id AS BIGINT)
        GROUP BY TRY_CAST(p1.department_id AS BIGINT), TRY_CAST(p2.department_id AS BIGINT), op1.product_id, op2.product_id
        HAVING COUNT(DISTINCT op1.order_id) >= {min_support}
    ),
    product_support AS (
        SELECT 
            product_id,
            COUNT(DISTINCT order_id) as support
        FROM workspace.instacart.order_products_prior
        GROUP BY product_id
    ),
    total_orders AS (
        SELECT COUNT(DISTINCT order_id) as total 
        FROM workspace.instacart.order_products_prior
    )
    SELECT 
        d1.department as department_1,
        d2.department as department_2,
        p1.product_name as product_1,
        p2.product_name as product_2,
        dp.pair_count,
        ROUND(dp.pair_count * 1.0 / ps1.support, 4) as confidence,
        ROUND(dp.pair_count * 1.0 * to.total / (ps1.support * ps2.support), 2) as lift
    FROM dept_pairs dp
    CROSS JOIN total_orders to
    LEFT JOIN workspace.instacart.departments d1 ON dp.dept1_id = d1.department_id
    LEFT JOIN workspace.instacart.departments d2 ON dp.dept2_id = d2.department_id
    JOIN workspace.instacart.products p1 ON dp.prod1 = p1.product_id
    JOIN workspace.instacart.products p2 ON dp.prod2 = p2.product_id
    JOIN product_support ps1 ON dp.prod1 = ps1.product_id
    JOIN product_support ps2 ON dp.prod2 = ps2.product_id
    WHERE ROUND(dp.pair_count * 1.0 * to.total / (ps1.support * ps2.support), 2) > {min_lift}
    ORDER BY lift DESC
    LIMIT 100
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("cross_department_insights")
    return df


# ============================================
# PART 6: SEQUENTIAL SHOPPING BEHAVIOR
# ============================================

def analyze_sequential_patterns(min_support=80):
    """
    Order in which products are added to cart
    """
    print("\n→ Mining sequential patterns...")
    
    query = f"""
    WITH ordered_products AS (
        SELECT 
            order_id,
            product_id,
            add_to_cart_order,
            ROW_NUMBER() OVER (PARTITION BY order_id ORDER BY add_to_cart_order) as position
        FROM workspace.instacart.order_products_prior
    ),
    sequences AS (
        SELECT 
            op1.product_id as first_product,
            op2.product_id as second_product,
            COUNT(*) as sequence_count,
            AVG(op1.position) as avg_first_position,
            AVG(op2.position) as avg_second_position
        FROM ordered_products op1
        JOIN ordered_products op2 
            ON op1.order_id = op2.order_id 
            AND op2.position = op1.position + 1
        GROUP BY op1.product_id, op2.product_id
        HAVING COUNT(*) >= {min_support}
    )
    SELECT 
        p1.product_name as first_product,
        p2.product_name as second_product,
        s.sequence_count,
        ROUND(s.avg_first_position, 1) as avg_first_pos,
        ROUND(s.avg_second_position, 1) as avg_second_pos,
        d1.department as first_dept,
        d2.department as second_dept,
        a1.aisle as first_aisle,
        a2.aisle as second_aisle
    FROM sequences s
    JOIN workspace.instacart.products p1 ON s.first_product = p1.product_id
    JOIN workspace.instacart.products p2 ON s.second_product = p2.product_id
    LEFT JOIN workspace.instacart.departments d1 ON TRY_CAST(p1.department_id AS BIGINT) = d1.department_id
    LEFT JOIN workspace.instacart.departments d2 ON TRY_CAST(p2.department_id AS BIGINT) = d2.department_id
    LEFT JOIN workspace.instacart.aisles a1 ON TRY_CAST(p1.aisle_id AS BIGINT) = a1.aisle_id
    LEFT JOIN workspace.instacart.aisles a2 ON TRY_CAST(p2.aisle_id AS BIGINT) = a2.aisle_id
    ORDER BY sequence_count DESC
    LIMIT 100
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("sequential_patterns")
    return df


# ============================================
# PART 7: BASKET DIVERSITY ANALYSIS
# ============================================

def analyze_basket_compositions():
    """
    Basket size and diversity metrics
    """
    print("\n→ Analyzing basket compositions...")
    
    query = """
    WITH basket_stats AS (
        SELECT 
            o.order_id,
            COUNT(DISTINCT op.product_id) as total_items,
            COUNT(DISTINCT p.department_id) as unique_departments,
            COUNT(DISTINCT p.aisle_id) as unique_aisles
        FROM workspace.instacart.orders o
        JOIN workspace.instacart.order_products_prior op ON o.order_id = op.order_id
        JOIN workspace.instacart.products p ON op.product_id = p.product_id
        WHERE o.eval_set = 'prior'
          AND TRY_CAST(p.department_id AS BIGINT) IS NOT NULL
          AND TRY_CAST(p.aisle_id AS BIGINT) IS NOT NULL
        GROUP BY o.order_id
    ),
    size_categories AS (
        SELECT 
            CASE 
                WHEN total_items <= 5 THEN 'Small (1-5 items)'
                WHEN total_items <= 15 THEN 'Medium (6-15 items)'
                WHEN total_items <= 30 THEN 'Large (16-30 items)'
                ELSE 'XLarge (30+ items)'
            END as basket_size,
            total_items,
            unique_departments,
            unique_aisles
        FROM basket_stats
    )
    SELECT 
        basket_size,
        COUNT(*) as basket_count,
        ROUND(AVG(total_items), 2) as avg_items,
        ROUND(AVG(unique_departments), 2) as avg_departments,
        ROUND(AVG(unique_aisles), 2) as avg_aisles,
        ROUND(AVG(unique_aisles * 1.0 / total_items), 3) as diversity_ratio,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as pct_of_total
    FROM size_categories
    GROUP BY basket_size
    ORDER BY 
        CASE basket_size
            WHEN 'Small (1-5 items)' THEN 1
            WHEN 'Medium (6-15 items)' THEN 2
            WHEN 'Large (16-30 items)' THEN 3
            ELSE 4
        END
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("basket_compositions")
    return df


# ============================================
# PART 8: REORDER LOYALTY PATTERNS
# ============================================

def analyze_reorder_patterns():
    """
    Products frequently reordered together show strong loyalty
    """
    print("\n→ Analyzing reorder patterns...")
    
    query = """
    WITH reordered_pairs AS (
        SELECT 
            op1.product_id as prod1,
            op2.product_id as prod2,
            COUNT(DISTINCT op1.order_id) as reorder_together_count
        FROM workspace.instacart.order_products_prior op1
        JOIN workspace.instacart.order_products_prior op2 
            ON op1.order_id = op2.order_id 
            AND op1.product_id < op2.product_id
        WHERE op1.reordered = 1 AND op2.reordered = 1
        GROUP BY op1.product_id, op2.product_id
        HAVING COUNT(DISTINCT op1.order_id) >= 100
    )
    SELECT 
        p1.product_name as product_1,
        p2.product_name as product_2,
        rp.reorder_together_count,
        d1.department as dept_1,
        d2.department as dept_2,
        CASE 
            WHEN d1.department = d2.department THEN 'Same Department Loyalty'
            ELSE 'Cross Department Loyalty'
        END as loyalty_type
    FROM reordered_pairs rp
    JOIN workspace.instacart.products p1 ON rp.prod1 = p1.product_id
    JOIN workspace.instacart.products p2 ON rp.prod2 = p2.product_id
    LEFT JOIN workspace.instacart.departments d1 ON TRY_CAST(p1.department_id AS BIGINT) = d1.department_id
    LEFT JOIN workspace.instacart.departments d2 ON TRY_CAST(p2.department_id AS BIGINT) = d2.department_id
    WHERE d1.department IS NOT NULL AND d2.department IS NOT NULL
    ORDER BY reorder_together_count DESC
    LIMIT 100
    """
    
    df = spark.sql(query)
    df.createOrReplaceTempView("reorder_patterns")
    return df


# ============================================
# MAIN EXECUTION
# ============================================

print("\n" + "="*70)
print("EXECUTING COMPREHENSIVE ANALYSIS")
print("="*70 + "\n")

results = {}

# Run all analyses
results['pairs'] = analyze_product_pairs(min_support_count=100)
results['triplets'] = analyze_triplets(min_support_count=50)
results['quadruplets'] = analyze_quadruplets(min_support_count=30)
results['temporal'] = analyze_temporal_patterns(min_support=50)
results['cross_dept'] = analyze_cross_department_synergies(min_lift=2.5)
results['sequences'] = analyze_sequential_patterns(min_support=80)
results['compositions'] = analyze_basket_compositions()
results['reorders'] = analyze_reorder_patterns()

print("\n" + "="*70)
print("✓ ANALYSIS COMPLETE!")
print("="*70)
print("\nAll temporary views created:")
print("  • product_pairs_analysis")
print("  • triplet_patterns")
print("  • quadruplet_patterns")
print("  • temporal_patterns")
print("  • cross_department_insights")
print("  • sequential_patterns")
print("  • basket_compositions")
print("  • reorder_patterns")

# ============================================
# DISPLAY KEY RESULTS
# ============================================

print("\n\n" + "="*70)
print("📊 KEY FINDINGS - TOP PATTERNS")
print("="*70)

print("\n\n🔥 TOP 15 HIGH-LIFT PRODUCT PAIRS (Strongest Associations):")
display(results['pairs'].orderBy(col('lift').desc()).limit(15))

print("\n\n🎯 TOP 15 TRIPLET PATTERNS (3-Item Combos):")
display(results['triplets'].orderBy(col('triplet_count').desc()).limit(15))

print("\n\n💎 TOP 10 QUADRUPLET PATTERNS (4-Item Complete Sets):")
display(results['quadruplets'].orderBy(col('quad_count').desc()).limit(10))

print("\n\n🌉 CROSS-DEPARTMENT DISCOVERIES (Unexpected Pairings):")
display(results['cross_dept'].orderBy(col('lift').desc()).limit(15))

print("\n\n⏰ WEEKEND EVENING SHOPPING PATTERNS:")
display(spark.sql("""
    SELECT * FROM temporal_patterns 
    WHERE day_type = 'Weekend' AND time_period = 'Evening (6-9pm)'
    ORDER BY pair_count DESC
    LIMIT 15
"""))

print("\n\n🔄 REORDER LOYALTY PATTERNS (Habitual Purchases):")
display(results['reorders'].orderBy(col('reorder_together_count').desc()).limit(15))

print("\n\n📦 BASKET SIZE DISTRIBUTION:")
display(results['compositions'])

print("\n\n" + "="*70)
print("Analysis complete! Use the temp views for further exploration.")
print("="*70)